In [94]:
import requests
from bs4 import BeautifulSoup
import urllib
import time
from retry import retry
import pandas as pd
import numpy as np

In [ ]:
# 音を出す
def sound():
    !rundll32 user32.dll,MessageBeep

In [100]:
# 検索ページのurlから物件の詳細を拾ってきたい

# 情報の器
data_samples = []

# 検索出力ページ（東京23区）
base_url = 'https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13101&sc=13102&sc=13103&sc=13104&sc=13105&sc=13113&sc=13106&sc=13107&sc=13108&sc=13118&sc=13121&sc=13122&sc=13123&sc=13109&sc=13110&sc=13111&sc=13112&sc=13114&sc=13115&sc=13120&sc=13116&sc=13117&sc=13119&cb=0.0&ct=9999999&mb=0&mt=9999999&et=9999999&cn=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sngz=&po1=25&pc=50&page={}'

# 拾うページ数
max_page = 2902

# ページ読み込み関数（リトライ機能付き）
@retry(tries=3, delay=10, backoff=2)
def load_page(load_url):
    html = requests.get(load_url)
    soup = BeautifulSoup(html.content, 'html.parser')
    return soup

# 処理時間を測りたい
start = time.time()
times = []

# ページごとの処理
for page in range(1,max_page+1):
    before = time.time()

    # ページ遷移
    load_url = base_url.format(page)
    # 検索ページ情報
    soup = load_page(load_url)
    print(page)
    print(load_url)

    # 検索ページから物件詳細の絶対urlを獲得
    for j, element in enumerate(soup.find_all(class_='js-cassette_link_href cassetteitem_other-linktext')):
        get_url = element.get('href')
        abs_url = urllib.parse.urljoin(load_url,get_url)
        soup_room = load_page(abs_url)

        # 部屋情報の器
        data_room = []

        # 物件名
        name = soup_room.find(class_='section_h1-header-title').text
        data_room.append(name)
        # 費用に関する情報
        colname = ['家賃','管理費・共益費','敷金','礼金','保証金','敷引・償却']
        cost_info = soup_room.find_all(class_='property_view_note-list')
        for mother in cost_info:
            for child in mother.find_all('span'):
                data_room.append(child.text)
        # 基本情報
        colname = ['住所','駅徒歩','間取り','面積','築年数','階','向き','建物種別']
        basic_info = soup_room.find_all(class_='property_view_table-body')
        for k, (name, info ) in enumerate(zip(colname,basic_info)):
            if k == 1:
                for child in info.find_all('div'):
                    data_room.append(child.text)
            else:
                data_room.append(info.text)
        # オプション情報
        option = soup_room.find_all(class_='inline_list')
        data_room.append(option[1].text)
        # 概要情報
        colname = ['間取り詳細','構造','階建','築年月','損保','駐車場','入居','取引態様','条件','取り扱い店舗物件コード','SUUMO物件コード','総戸数','情報更新日','次回更新日','仲介手数料','保証会社','ほか初期費用']
        overview = soup_room.find(class_='data_table table_gaiyou')
        children = overview.find_all('td')
        for name, info in zip(colname,children):
            data_room.append(info.text.strip())
            
        # url情報
        data_room.append(abs_url)

        # 器に部屋情報を入れる
        data_samples.append(data_room)

        # requestsしたら1秒休む
        time.sleep(1)
        
    # 進捗確認
    # このページの作業時間を表示
    after = time.time()
    running_time = after - before
    times.append(running_time)
    print(f'{page}ページ目：{running_time}秒')
    # 取得した件数
    print(f'総取得件数：{len(data_samples)}')
    # 作業進捗
    complete_ratio = round(page/max_page*100,3)
    print(f'完了：{complete_ratio}%')
    # 作業の残り時間目安を表示
    running_mean = np.mean(times)
    running_required_time = running_mean * (max_page - page)
    hour = int(running_required_time/3600)
    minute = int((running_required_time%3600)/60)
    second = int(running_required_time%60)
    print(f'残り時間：{hour}時間{minute}分{second}秒\n')

# 処理時間を測りたい
finish = time.time()
running_all = finish - start
print('総経過時間：',running_all)
sound()

1
https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13101&sc=13102&sc=13103&sc=13104&sc=13105&sc=13113&sc=13106&sc=13107&sc=13108&sc=13118&sc=13121&sc=13122&sc=13123&sc=13109&sc=13110&sc=13111&sc=13112&sc=13114&sc=13115&sc=13120&sc=13116&sc=13117&sc=13119&cb=0.0&ct=9999999&mb=0&mt=9999999&et=9999999&cn=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sngz=&po1=25&pc=50&page=1
1ページ目：245.4734661579132秒
総取得件数：124
完了：0.034%
残り時間：197時間48分38秒

2
https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13101&sc=13102&sc=13103&sc=13104&sc=13105&sc=13113&sc=13106&sc=13107&sc=13108&sc=13118&sc=13121&sc=13122&sc=13123&sc=13109&sc=13110&sc=13111&sc=13112&sc=13114&sc=13115&sc=13120&sc=13116&sc=13117&sc=13119&cb=0.0&ct=9999999&mb=0&mt=9999999&et=9999999&cn=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sngz=&po1=25&pc=50&page=2


KeyboardInterrupt: 

In [ ]:
colname = ['物件名','家賃','管理費・共益費','敷金','礼金','保証金','敷引・償却','住所','駅徒歩1','駅徒歩2','駅徒歩3','間取り','面積','築年数','階','向き','建物種別','オプション','間取り詳細','構造','階建','築年月','損保','駐車場','入居','取引態様','条件','取り扱い店舗物件コード','SUUMO物件コード','総戸数','情報更新日','次回更新日','仲介手数料','保証会社','ほか初期費用','url']
pd.DataFrame(data_samples, columns=colname)['url'][0]